# ■ 음악 장르 분류기 (Tensorflow 작성)

- ### 교육자료(28페이지 부터)
  - https://www.dropbox.com/s/j104afvzutjas25/30.%20Deep%20Learning%20%EC%9D%B4%EB%A1%A0%28DNN_CNN_RNN%29.pdf
- ### 데이터셋
  - (http://marsyasweb.appspot.com/download/data_sets/)
- #### 음원 라이브러리가 작동하지 않을때 적용가능 해결법들
  - (https://stackoverflow.com/questions/2060628/reading-wav-files-in-python)

In [24]:
from pylab import *
import numpy as np
import scipy.signal
import scipy.fftpack
from scipy.io import wavfile as wvf # 이거 안되네 슈...
from scipy.fftpack import dct
import os
import decimal # 특이한 라이브러리 (반올림 함수짤때 사용)

# wavfile로 안읽어진다 ㅜㅜ 따라서 다른방법 시도
import pyaudio
import soundfile as sf # 이걸 사용하자.

## wav파일-->Array변환, Y값 라벨링

In [117]:
# Y값 수집 및 처리
y_list = []
for wav_label in wav_labels:
    print(wav_label,end=",")
    for wav_f in os.listdir(wav_dir+wav_label):
        y_list.append(wav_label)
        
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = y_list
le.fit(y)
y= le.transform(y)
y[:100]

blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock,

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [118]:
# X값 데이터 수집
# WAV 파일 읽은 후 Array 형태로 변환합니다.
wav_dir = 'D:/d_sound_datas/GTZAN_genre_collection/'

wav_labels = os.listdir(wav_dir)
x = []

print(wav_labels[:],end="")
for wav_label in wav_labels:
    print(wav_label,end="\n")
    for wav_f in os.listdir(wav_dir+wav_label):
        print("-->%s "%wav_f, end = "               \r")
####작동안되서 폐기--> SampleFreq, f = wvf.read(wav_dir+wav_label+'/' +  wav_f)
        f, SampleFreq = sf.read(wav_dir+wav_label+'/' +  wav_f) 
        N_total = len(f)
        N_cut = SampleFreq*5
        audio_data = f[N_cut:N_total-N_cut]
        N = len(audio_data)
        x.append(audio_data)
#         if wav_label == 'pop':
#             y.append([1,0,0,0])
#         elif wav_label == 'classical':
#             y.append([0,1,0,0])
#         elif wav_label == 'jazz':
#             y.append([0,0,1,0])
#         else :
#             y.append([0,0,0,1])
    print("                     \r--> ok!")
music = np.array(x) / (2.**15)
label = np.array(y)

['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']blues
--> ok!                                                                                                                    
classical
--> ok!                                                                                                                                                                                                   
country
--> ok!                                                                                                                                                    
disco
--> ok!                                                                                      
hiphop
--> ok!                                                                                                                                                   
jazz
--> ok!                                                                                                                             

## 전처리코드
- 스피치 전저리코드 설명(mfcc)
  - https://github.com/jameslyons/python_speech_features

In [59]:
from python_speech_features import mfcc, logfbank

def round_half_up(number):
    return int(decimal.Decimal(number).quantize(decimal.Decimal('1'), rounding=decimal.ROUND_HALF_UP))

#MFCC 추출을 위한 파라미터 값을 정의합니다.
samplerate = SampleFreq
winlen = 0.02
winstep = 0.01
numcep = 15
nfilt = 26
frame_len = int(round_half_up(winlen * samplerate))
nfft = frame_len
lowfreq = 0
highfreq = None
preemph = 0.97
ceplifter = 22
appendEnergy = True
winfunc = lambda x:np.hamming(x)

dnn_feat = []

def normalize(a):
    b = (a-np.mean(a))/np.std(a)
    return b

# MFCC를 추출
print("전처리할 항목 갯수 :",len(music))
for i in arange(len(music)):
    mfcc_feat = mfcc(music[i], samplerate, winlen, winstep, numcep, nfilt, nfft, lowfreq, highfreq, preemph, ceplifter, appendEnergy, winfunc)
    print("Extracting MFCC.. now step %d"%(i),end="\r")
    mfcc_feat = np.apply_along_axis(normalize,1,mfcc_feat)
    feat_mean = np.mean(mfcc_feat, axis = 0)
    feat_cov = np.cov(mfcc_feat,rowvar=0)
    feat_cov_up = np.triu(feat_cov, k=0)
    feat_flat = feat_cov_up.flatten()
    index = arange(len(feat_flat))[feat_flat==0]
    feat_cov = list(np.delete(feat_flat,index))
    feat = []
    feat.extend(feat_mean)
    feat.extend(feat_cov)
    dnn_feat.append(feat)
    
dnn_feat = np.array(dnn_feat)
print("\rExtracting MFCC : Complete!")

전처리할 항목 갯수 :  1000
Extracting MFCC : Complete!999


# TensorFlow 코드

In [141]:
# 데이터셋제작 및  분류

N_music = len(dnn_feat)
N_train = int(ceil(N_music * 0.70))
x_datas = dnn_feat
y_datas = y

# N_music = len(dnn_feat)
# N_train = int(ceil(N_music * 0.70))
# idx = np.random.choice(N_music, N_train, replace=False)
# x_train = dnn_feat[idx]
# y_train = label[idx]

# test_idx = np.delete(arange(N_music),idx)
# x_test = dnn_feat[test_idx]
# y_test = label[test_idx]


from sklearn.model_selection import train_test_split
x_train, x_test, y_train_tmp, y_test_tmp = train_test_split(x_datas, y_datas, test_size=0.3, random_state=0)

from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train_tmp, 10)
y_test = np_utils.to_categorical(y_test_tmp, 10)

In [170]:
# DNN구축
import tensorflow as tf

num_feat = len(dnn_feat[0,:])
num_classes = len(set(label))

x = tf.placeholder(tf.float32, [None, num_feat])
y_ = tf.placeholder(tf.float32, [None, num_classes])

### 학습 레이어정의

In [171]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0,1,shape=shape)
    return tf.Variable(initial)

num_neuron = 25

# 첫번째 레이어
W_1 = weight_variable([num_feat, num_neuron])
b_1 = bias_variable([num_neuron])

# 드랍아웃 정의
keep_prob = tf.placeholder("float")
W_1_drop = tf.nn.dropout(W_1, keep_prob)

# Relu 함수 정의
h_1 = tf.nn.relu(tf.matmul(x,W_1_drop)+b_1)

# 두번째 레이어
W_2 = weight_variable([num_neuron, num_classes])
b_2 = bias_variable([num_classes])

y = tf.nn.softmax(tf.matmul(h_1,W_2)+b_2)

### 학습하기
- 결론 : 드롭아웃을위해 모델훈련간 0.5의확률을 지정, 모델평가결과 정확도는 약93%로 높은 수치를 기록했음을 알수 있습니다.

In [ ]:
# 학습 초기화
import time
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session()
sess.run(tf.initialize_all_variables())

In [174]:
print("=============== Session start! ==================")
walltime = time.time()
epoch_time = walltime
for i in range(10000):
    if i%100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x: x_train, y_:y_train, keep_prob:1.0})
        test_accuracy = sess.run(accuracy,{x: x_test, y_:y_test, keep_prob:1.0})
        print("step %4d, %.2fsec, acc : %.4f"%(i, time.time()-epoch_time, train_accuracy), end=", ")
        train_cost = np.exp(sess.run(cross_entropy, feed_dict={x: x_train, y_:y_train, keep_prob:1.0}))
        print("cost : %.4f, val_acc : %.4f"%(train_cost,test_accuracy))
        epoch_time = time.time()
    #본 훈련!
    sess.run(train_step, {x: x_train, y_:y_train, keep_prob:0.6})

print("test accuracy %.4f"% sess.run(accuracy,{x: x_test, y_:y_test, keep_prob:1.0}))
print("wall time : %d sec"%(time.time()-walltime))

=============== Session start! ==================
step    0, 0.01sec, acc : 0.8471, cost : 1.7195, val_acc : 0.6533
step  100, 0.35sec, acc : 0.8471, cost : 1.7180, val_acc : 0.6667
step  200, 0.34sec, acc : 0.8443, cost : 1.7156, val_acc : 0.6567
step  300, 0.42sec, acc : 0.8529, cost : 1.7117, val_acc : 0.6667
step  400, 0.41sec, acc : 0.8457, cost : 1.7114, val_acc : 0.6700
step  500, 0.33sec, acc : 0.8486, cost : 1.7079, val_acc : 0.6633
step  600, 0.35sec, acc : 0.8457, cost : 1.7058, val_acc : 0.6667
step  700, 0.41sec, acc : 0.8457, cost : 1.7037, val_acc : 0.6667
step  800, 0.36sec, acc : 0.8514, cost : 1.7019, val_acc : 0.6667
step  900, 0.35sec, acc : 0.8529, cost : 1.6994, val_acc : 0.6667
step 1000, 0.39sec, acc : 0.8486, cost : 1.6977, val_acc : 0.6667
step 1100, 0.34sec, acc : 0.8500, cost : 1.6958, val_acc : 0.6633
step 1200, 0.32sec, acc : 0.8514, cost : 1.6943, val_acc : 0.6567
step 1300, 0.32sec, acc : 0.8529, cost : 1.6914, val_acc : 0.6667
step 1400, 0.33sec, acc : 

In [175]:
# y_pred = y
y_pred = sess.run(tf.argmax(y,1),feed_dict={x: x_test, keep_prob:1.0})
y_pred

array([9, 8, 0, 5, 6, 8, 0, 2, 9, 7, 4, 5, 7, 3, 5, 0, 0, 1, 0, 0, 0, 0,
       3, 9, 0, 7, 0, 5, 7, 9, 3, 2, 6, 7, 4, 7, 3, 3, 9, 0, 1, 9, 6, 6,
       2, 2, 8, 1, 3, 5, 8, 2, 2, 3, 9, 3, 2, 3, 6, 8, 5, 6, 1, 7, 8, 1,
       9, 5, 2, 5, 7, 2, 2, 2, 9, 3, 3, 7, 6, 6, 2, 4, 9, 3, 5, 6, 2, 5,
       4, 4, 8, 4, 9, 2, 8, 4, 5, 1, 3, 6, 7, 4, 0, 8, 3, 6, 6, 4, 3, 7,
       2, 8, 4, 5, 0, 4, 5, 3, 4, 5, 7, 9, 1, 2, 0, 8, 3, 2, 7, 9, 6, 6,
       5, 2, 5, 6, 8, 0, 5, 8, 1, 6, 5, 2, 0, 8, 6, 3, 0, 0, 2, 9, 4, 7,
       9, 1, 4, 3, 8, 4, 5, 3, 0, 8, 9, 3, 9, 4, 6, 4, 1, 7, 1, 8, 4, 7,
       4, 6, 3, 1, 7, 9, 4, 3, 0, 9, 7, 4, 7, 2, 8, 8, 4, 0, 5, 6, 7, 2,
       6, 1, 6, 1, 8, 9, 8, 6, 8, 5, 5, 3, 1, 6, 5, 2, 4, 6, 1, 2, 0, 5,
       0, 5, 0, 2, 4, 2, 6, 5, 0, 3, 0, 8, 6, 7, 0, 6, 6, 1, 7, 1, 5, 8,
       3, 3, 6, 4, 3, 0, 4, 6, 3, 6, 0, 0, 8, 7, 3, 4, 0, 0, 7, 7, 8, 1,
       8, 6, 7, 4, 5, 4, 3, 5, 0, 1, 8, 2, 2, 5, 8, 1, 9, 4, 9, 8, 2, 0,
       4, 8, 2, 2, 6, 2, 7, 9, 7, 1, 9, 3, 4, 5], d

In [176]:
le.inverse_transform(y_pred)

C:\Users\creal\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['rock', 'reggae', 'blues', 'jazz', 'metal', 'reggae', 'blues',
       'country', 'rock', 'pop', 'hiphop', 'jazz', 'pop', 'disco', 'jazz',
       'blues', 'blues', 'classical', 'blues', 'blues', 'blues', 'blues',
       'disco', 'rock', 'blues', 'pop', 'blues', 'jazz', 'pop', 'rock',
       'disco', 'country', 'metal', 'pop', 'hiphop', 'pop', 'disco',
       'disco', 'rock', 'blues', 'classical', 'rock', 'metal', 'metal',
       'country', 'country', 'reggae', 'classical', 'disco', 'jazz',
       'reggae', 'country', 'country', 'disco', 'rock', 'disco',
       'country', 'disco', 'metal', 'reggae', 'jazz', 'metal',
       'classical', 'pop', 'reggae', 'classical', 'rock', 'jazz',
       'country', 'jazz', 'pop', 'country', 'country', 'country', 'rock',
       'disco', 'disco', 'pop', 'metal', 'metal', 'country', 'hiphop',
       'rock', 'disco', 'jazz', 'metal', 'country', 'jazz', 'hiphop',
       'hiphop', 'reggae', 'hiphop', 'rock', 'country', 'reggae',
       'hiphop', 'jazz',